In [25]:
import matplotlib.pyplot as plt
import matplotlib
import datetime as dt
from glob import glob
from tqdm import tqdm
import json
import pandas as pd
#import seaborn as sns
import csv
import re
import numpy as np

from collections import Counter
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

def draw_graph(project, positive_metrics_lists, negative_metrics_lists, metrics_list):
    matplotlib.rcParams['font.family'] = 'Hiragino Sans'  # Macで日本語対応

    fig, axes = plt.subplots(5, 3, figsize=(18, 24))
    axes = axes.flatten()

    stats_rows = []
    log_metrics = ["報告実績", "経過時間", "レビュー行数"]
    bar_metrics = ["テストコード含有", "バグ修正確信度", "リファクタリング"]

    # カテゴリごとの色
    color_map = {0: "red", 1: "green", 2: "blue"}

    for i, metrics in enumerate(metrics_list):
        pos = positive_metrics_lists[i]
        neg = negative_metrics_lists[i]
        ax = axes[i]
    
        if metrics in bar_metrics:
            # --- 積み上げ棒グラフ（割合） ---
            categories = sorted(set(pos) | set(neg))
            pos_count = Counter(pos)
            neg_count = Counter(neg)
    
            total_pos = sum(pos_count.values()) or 1
            total_neg = sum(neg_count.values()) or 1
    
            bottom_pos, bottom_neg = 0, 0
            bars_for_legend = []
    
            for c in categories:
                pos_height = pos_count[c] / total_pos
                neg_height = neg_count[c] / total_neg
    
                b1 = ax.bar("Positive", pos_height, bottom=bottom_pos, color=color_map.get(c, "gray"))
                b2 = ax.bar("Negative", neg_height, bottom=bottom_neg, color=color_map.get(c, "gray"))
    
                bottom_pos += pos_height
                bottom_neg += neg_height
                bars_for_legend.append(b1)
    
            ax.set_ylim(0, 1)
            ax.set_title(metrics, fontsize=12)
            ax.legend(bars_for_legend, [f"Value {c}" for c in categories])
    
        else:
            # --- 箱ひげ図 ---
            ax.boxplot([pos, neg], labels=["Positive", "Negative"], sym="")
            ax.set_title(metrics, fontsize=12)
            if metrics in log_metrics:
                ax.set_yscale('log')

        # 四分位数の計算
        for label, data in [("Positive", pos), ("Negative", neg)]:
            if len(data) > 0:
                q1 = np.percentile(data, 25)
                med = np.percentile(data, 50)
                q3 = np.percentile(data, 75)
            else:
                q1 = med = q3 = None

            stats_rows.append({
                "metric": metrics,
                "type": label,
                "Q1": q1,
                "Median": med,
                "Q3": q3
            })

    # レイアウト調整・保存
    plt.tight_layout()
    plt.savefig(f"/Users/mizuki-u/current/論文誌用/{project}/all_metrics_boxplots.png", transparent=True)
    plt.close()

    # CSV出力
    stats_df = pd.DataFrame(stats_rows)
    stats_df.to_csv(f"/Users/mizuki-u/current/論文誌用/{project}/quartiles.csv", index=False)




def positive_metrics(project, metrics_list, window_num_least, window_num_most):
    metrics_lists = [[] for _ in range(15)]

    with open(f'/Users/mizuki-u/current/backup/shuron_result/修正中_予測/metrics_list/review/{project}/propose_True_Positives.csv') as f:
        reader = csv.DictReader(f)
        line = [row for row in reader]
    for l in line:
        if window_num_least <= int(l['期間']) <= window_num_most:
            for i, metrics in enumerate(metrics_list):
                if i <= 1:
                    metrics_lists[i].append(float(l[metrics]))
                else:
                    metrics_lists[i].append(int(l[metrics]))
    
    with open(f'/Users/mizuki-u/current/backup/shuron_result/修正中_予測/metrics_list/review/{project}/propose_False_Negatives.csv') as f:
        reader = csv.DictReader(f)
        line = [row for row in reader]
    for l in line:
        if window_num_least <= int(l['期間']) <= window_num_most:
            for i, metrics in enumerate(metrics_list):
                if i <= 1:
                    metrics_lists[i].append(float(l[metrics]))
                else:
                    metrics_lists[i].append(int(l[metrics]))

    return metrics_lists


def negative_metrics(project, metrics_list, window_num_least, window_num_most):
    metrics_lists = [[] for _ in range(15)]

    with open(f'/Users/mizuki-u/current/backup/shuron_result/修正中_予測/metrics_list/review/{project}/propose_False_Positives.csv') as f:
        reader = csv.DictReader(f)
        line = [row for row in reader]
    for l in line:
        if window_num_least <= int(l['期間']) <= window_num_most:
            for i, metrics in enumerate(metrics_list):
                if i <= 1:
                    metrics_lists[i].append(float(l[metrics]))
                else:
                    metrics_lists[i].append(int(l[metrics]))
    
    with open(f'/Users/mizuki-u/current/backup/shuron_result/修正中_予測/metrics_list/review/{project}/propose_True_Negatives.csv') as f:
        reader = csv.DictReader(f)
        line = [row for row in reader]
    for l in line:
        if window_num_least <= int(l['期間']) <= window_num_most:
            for i, metrics in enumerate(metrics_list):
                if i <= 1:
                    metrics_lists[i].append(float(l[metrics]))
                else:
                    metrics_lists[i].append(int(l[metrics]))

    return metrics_lists


project_list = ['Nova', 'Neutron', 'Keystone', 'Swift']
metrics_list = ['導入実績', '直近導入実績', '報告実績', '直近報告実績', '追加行数', '削除行数', 'ファイル数', '現リビジョン', '経過時間', 'テストコード含有', 'バグ修正確信度', 'リファクタリング', '期間', 'PR数', 'レビュー行数']
for project in project_list:
    if project == 'Nova':
        window_num_least = 20
        window_num_most = 34
    elif project == 'Neutron':
        window_num_least = 14
        window_num_most = 28
    elif project == 'Keystone':
        window_num_least = 27
        window_num_most = 41
    elif project == 'Swift':
        window_num_least = 20
        window_num_most = 34

    positive_metrics_lists = positive_metrics(project, metrics_list, window_num_least, window_num_most)
    negative_metrics_lists = negative_metrics(project, metrics_list, window_num_least, window_num_most)

    draw_graph(project, positive_metrics_lists, negative_metrics_lists, metrics_list)